# Set up DFs

Set up Model and Beta DFs

In [1]:
%matplotlib inline
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

if sys.platform == 'darwin':
    cfg = os.path.join("/Users", "njchiang", "CloudStation", "Grad",
                       "Research", "montilab-ucla", "analogy", "config", "project.json")
    plat = "osx"

elif sys.platform == "linux":
    import platform
    if platform.linux_distribution()[0] == "debian":
        cfg = os.path.join("/home", "njchiang", "data", "CloudStation", "Grad",
                           "Research", "montilab-ucla", "analogy", "config", "project.json")
        plat = "linux"
    else:
        cfg = os.path.join("/u", "project", "monti", "Analysis", "Analogy",
                           "code", "analogy", "config", "project.json")
        plat = "hoff"
else:
    cfg = os.path.join("D:\\", "CloudStation", "Grad",
                       "Research", "montilab-ucla", "analogy", "config", "project.json")
    plat = "win"

with open(cfg, "r") as f:
    projectSettings = json.load(f)

paths = projectSettings["filepaths"]["{}Paths".format(plat)]
sys.path.append(paths["github"])
sys.path.append(paths["code"])


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
from fmri_core import analysis as pa
from fmri_core import utils as pu
from fmri_core import vis as pv


analysisSettings = pu.load_config(os.path.join(paths["code"], "config", "analyses.json"))
contrastSettings = pu.load_config(os.path.join(paths["code"], "config", "contrasts.json"))

Loading JSON config from D:\CloudStation\Grad\Research\montilab-ucla\analogy\config\analyses.json
Loading JSON config from D:\CloudStation\Grad\Research\montilab-ucla\analogy\config\contrasts.json


In [8]:
from fmri.analogy_utils import analysisSettings, contrastSettings, order, \
    pu, pa, pv, compile_models, rsa, \
    save_rois, load_rois

In [9]:
models = compile_models(write=False)


model_mat = []
names = []
model_mat.append(
    np.array(
        [models["rel"][v] for v in order.ABTag.values[::2]]
    )
)

names += ["mainrel_1", "mainrel_2", "mainrel_3", 
          "subrel_1", "subrel_2", "subrel_3", 
          "subrel_4", "subrel_5", "subrel_6", 
          "subrel_7", "subrel_8", "subrel_9"]

model_mat.append(
    np.array(
        [models["humanratings"][v] for v in order.ABTag.values[::2]]
    )
)

names += ["humanratings_1", "humanratings_2", "humanratings_3", 
          "humanratings_4", "humanratings_5", "humanratings_6", 
          "humanratings_7", "humanratings_8", "humanratings_9"]

model_mat.append(
    np.array(
        [models["w2vdiff"][v] for v in order.ABTag.values[::2]]
    )
)

names += ["w2vdiff_{}".format(i) for i in range(1, 301)]

model_mat.append(
    np.array(
        [models["rstpostprob9"][v] for v in order.ABTag.values[::2]]
    )
)

names += ["rstpostprob9_{}".format(i) for i in range(1, 10)]

model_mat.append(
    np.array(
        [models["rstpostprob79"][v] for v in order.ABTag.values[::2]]
    )
)

names += ["rstpostprob79_{}".format(i) for i in range(1, 80)]

model_mat.append(
    np.array(
        [models["concatword"][v] for v in order.ABTag.values[::2]]
    )
)

names += ["concatword_{}".format(i) for i in range(1, 601)]

model_mat.append(
    np.array(
        [models["typicality"][v] for v in order.ABTag.values[::2]]
    )
)

names += ["typicality"]

model_mat.append(
    np.array([
        [len(v)] for v in order.ABTag.values[::2]
    ])
)
    
names += ["numchar"]

models_df = pd.DataFrame(
    np.concatenate(model_mat, axis=1), 
    columns=names, 
    index=order.ABTag.values[::2]
)
models_df.join(
    order[::2][["ABTag", "ABMainRel", "ABSubRel"]]
    .set_index(keys="ABTag")
    .rename(columns={"ABMainRel": "MainCond", 
                     "ABSubRel": "SubCond"})).to_csv(
    os.path.join(paths["code"], "labels", "raw_models.csv"),
    index_label="ABTag"
)


Loading label file from: D:\CloudStation\Grad\Research\montilab-ucla\analogy\labels\typicality.csv
Loading label file from: D:\CloudStation\Grad\Research\montilab-ucla\analogy\labels\word2vec_diffs.csv
Loading label file from: D:\CloudStation\Grad\Research\montilab-ucla\analogy\labels\humanratings.csv
Loading mat file...
Loading mat file...
Loading mat file...


In [28]:
modelnames = ["rel", "humanratings", "w2vdiff", 
              "rstpostprob9", "rstpostprob79", 
              "numchar", "concatword", "typicality"]

In [45]:
models_df = pu.load_labels(os.path.join(paths["code"], "labels", "raw_models.csv"))

Loading label file from: D:\CloudStation\Grad\Research\montilab-ucla\analogy\labels\raw_models.csv


In [46]:
models_df.head()

,ABTag,mainrel_1,mainrel_2,mainrel_3,subrel_1,subrel_2,subrel_3,subrel_4,subrel_5,subrel_6,...,concatword_{}.594,concatword_{}.595,concatword_{}.596,concatword_{}.597,concatword_{}.598,concatword_{}.599,typicality,numchar,MainCond,SubCond
0,big:large,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.09082,-0.02490,-0.00510,-0.07227,0.03735,-0.15625,31.5,9.0,1,1
1,boat:ship,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.08643,-0.16211,-0.21191,-0.00067,0.02161,-0.15723,13.6,9.0,1,1
2,car:auto,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.11230,-0.02576,-0.04126,0.22852,-0.14941,-0.15039,35.7,8.0,1,1
3,careful:cautious,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.19336,-0.21680,0.17578,0.03467,-0.17188,-0.08398,24.1,16.0,1,1
4,couch:sofa,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.30469,-0.41016,0.00117,0.07959,0.08545,-0.00183,22.4,10.0,1,1
